In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
from string import Template
import os
import re
from collections import defaultdict
import random
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline


In [2]:
from datetime import datetime, timedelta

dateString = "2013/12/21"
dateFormatter = "%Y/%m/%d"
datetime.strptime(dateString, dateFormatter)
test_symbols = ['2330', '3008', '2317', '2412']
DEBUG = False

loop = asyncio.get_event_loop()

In [3]:
# async pipline Test brokerId fetch  
async def fetch_brokerId():
    URL = 'https://fubon-ebrokerdj.fbs.com.tw/z/js/zbrokerjs.djjs'
    
    resp = requests.get(URL)
    resp.encoding = 'cp950' # 中文
    return resp.text
  
async def post_brokerId():
    global BkrMap
    text =  await fetch_brokerId()
    reBkr = re.compile(r".*g_BrokerList = \'(.*)\'", re.MULTILINE|re.DOTALL)
    reBkr2Id = re.compile(r"(.*),(.*)", re.DOTALL)
    BkrGrps = reBkr.match(text).group(1).split(';') # slice to broker groups
    BkrMap = defaultdict(lambda: defaultdict(str))

    for bkrgrp in BkrGrps:
        bkrlist = bkrgrp.split('!') # slice to sub broker
        for i, bkr in enumerate(bkrlist):
            bmt = reBkr2Id.match(bkr)
            if bmt:
                top_bkr = bmt.group(2) if i == 0 else top_bkr
                BkrMap[top_bkr].update({bmt.group(2): bmt.group(1)}) #group(1) = b

    # debug            
    #for k in sorted(BkrMap.keys())[:1]:
    #    print(BkrMap[k])
loop = asyncio.get_event_loop()
loop.run_until_complete(post_brokerId())
#post_brokerId()

In [4]:
# random pick BHID, SBHIDS
def random_pick_BHIDs():
    global BkrMap
    test_BHIDS = list(BkrMap[random.choice(list(BkrMap.keys()))].items())
    print (test_BHIDS)
    BHID = test_BHIDS[0][1]
    SBHID = random.choice(test_BHIDS[1:])[1] if len(test_BHIDS) > 2 else BHID # if have sub SBHIDS
    #print (BHID, SBHID)
    return (BHID, SBHID)
    

In [5]:
BHID = '8150' # 台新
SBHID = '8151' # 台新建北

if not DEBUG:
    BHID, SBHID = random_pick_BHIDs()
    
print (BHID, SBHID)

[('永豐金', '9A00'), ('永豐金證券', '0039004100300030'), ('永豐金-三民(停)', '0039004100390056'), ('永豐金-三重', '003900410039004b'), ('永豐金-大園', '0039004100390038'), ('永豐金-大稻埕(停)', '0039004100390033'), ('永豐金-中正', '0039004100390055'), ('永豐金-中和', '0039004100390042'), ('永豐金-中盛', '0039004100390066'), ('永豐金-中壢', '0039004100390039'), ('永豐金-五權(停)', '0039004100360038'), ('永豐金-內湖', '0039004100390067'), ('永豐金-天母', '0039004100390047'), ('永豐金-北高雄', '0039004100390072'), ('永豐金-古亭', '0039004100390064'), ('永豐金-台中', '003900410039004c'), ('永豐金-台南', '0039004100390068'), ('永豐金-市政', '0039004100390057'), ('永豐金-永康', '0039004100390063'), ('永豐金-光復(停)', '0039004100390076'), ('永豐金-安和(停)', '0039004100390046'), ('永豐金-竹北', '0039004100390050'), ('永豐金-竹科', '0039004100390058'), ('永豐金-府城(停)', '0039004100390070'), ('永豐金-忠孝', '0039004100390044'), ('永豐金-板盛', '0039004100390059'), ('永豐金-板新', '003900410039004a'), ('永豐金-板橋', '0039004100390045'), ('永豐金-松山', '0039004100390031'), ('永豐金-虎尾', '0039004100390062'), ('永豐金-信義', '0039004100390052'), ('永

In [6]:
#http://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=2330&BHID=9800&b=9800
# get broker stock position detail put/call volume
def get_broker_sub_symbol_position(BHID, SBHID, symbol):
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=$SYMBOL&BHID=$BHID&b=$SBHID')
    URLPar = {
        'SYMBOL': symbol,
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 't01'})[0]
    values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
    ddates = list(map(lambda x: datetime.strptime(str(x.text), dateFormatter), table.find_all('td', {'class': 't4n0'})))
    #print (values)
    #print (len(values))
    calls = [values[i] for i in range(0, len(values), 4)]  #買
    puts = [values[i+1] for i in range(0, len(values), 4)] #賣
    BkrPutCall = {
        'date': ddates,
        'symbol': symbol,
        'puts': puts,
        'calls': calls
    }
    return  pd.DataFrame(BkrPutCall)

In [7]:
symbol = '2330'
if not DEBUG:
    symbol = random.choice(test_symbols)

BkrPutCallpf = get_broker_sub_symbol_position(BHID, SBHID, symbol)
BkrPutCallpf[:4]

,date,symbol,puts,calls
0,2021-02-17,2317,27,20
1,2021-02-05,2317,21,24
2,2021-02-04,2317,19,18
3,2021-02-03,2317,19,7


In [8]:
#https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=1440&b=1440&c=E&d=1
def get_broker_all_symbol_position(BHID, SBHID):
    
    # parser GenLink2stk('AS3481','群創');
    reSymbol2Id = re.compile(r".*GenLink2stk\(\'AS(.*)\',\'(.*)\'\);", re.M|re.DOTALL)  
    
    #parser <a href="javascript:Link2Stk('00677U');">00677U富邦VIX</a>
    # no support yet
    #reSymbol2Id2 = re.compile(r".*Link2Stk\(\'(.*)\'\);\"\>(.*)\<\/a\>", re.M|re.DOTALL)
    
    def get_symbol(text): 
        smt = reSymbol2Id.match(text)
        if smt:
            return smt.group(1)
            
    def get_puts_calls(table, Bkrlist):
        values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
        symbols = list(map(lambda x:  get_symbol(str(x.text)), table.find_all('td', {'class': 't4t1'})))
        
        for i, symbol in enumerate(symbols):
            if symbol == None:
                continue
            Bkrlist.append({
            'symbol': symbol,
            'calls': values[i*3+0], #買
            'puts': values[i*3+1]   #賣
            })
        
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=$BHID&b=$SBHID&c=E&d=1')
    URLPar = {
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    tables = soup.find_all('table', {'class': 't0'})
    Bkrlist = list()
    get_puts_calls(tables[0], Bkrlist) # 買超
    get_puts_calls(tables[1], Bkrlist) # 賣超
    
    return pd.DataFrame(Bkrlist)

In [9]:
BHID2Symbolpf = get_broker_all_symbol_position(BHID, SBHID)
BHID2Symbolpf.head()

,symbol,calls,puts
0,2603,504,348
1,2409,147,49
2,6282,77,1
3,6120,51,0
4,6208,52,1


In [10]:
from mongoengine import *

In [11]:
disconnect()
connect(alias='symbol-db-alias', db='symbol-db')
connect(alias='broker-db-alias', db='broker-db')
connect(alias='symbol-broker-db-alias', db='symbol-broker-db')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [12]:
#TopBroker = {
#  'BrokerId' : {
#  [
#       'date0': { [{'symbolId', 'puts', 'calls'}, ...]},
#]
#}
#}
class BrokerPositionDB(EmbeddedDocument): #embed
    symbol = IntField() # ref symbol to Symbol Top 
    puts = IntField()   # puts volume
    calls = IntField()  # call volume

class BrokerDB(Document):
    date = DateTimeField(default=datetime.utcnow)
    position = ListField(EmbeddedDocumentField(BrokerPositionDB))
    
    meta = {'db_alias': 'broker-db-alias'}
    
class BarDB(Document):
    O = FloatField()
    L = FloatField()
    V = IntField()
    
#MarginPS(Document):

class SymbolDB(Document):
    date = DateTimeField(default=datetime.utcnow)
    broker = ReferenceField(BrokerDB)
    bar = ReferenceField(BarDB)

    meta = {'db_alias': 'symbol-broker-db-alias'}
    

In [13]:
#bkrPost = BkrPutCallpf.iloc[0].to_dict()
BrkSymbolDic = BHID2Symbolpf.to_dict(orient="records")

In [14]:
# clean BrokerDB as empty
BrokerDB.drop_collection()

# random gen 10 broker symbols to mongodb
for i in range(10):
    brokerDb = BrokerDB()
    brokerDb.date = datetime.now() - timedelta(days=i)
    # pipeline from scrapy to MongoeEngine store
    brokerDb.position = [BrokerPositionDB(**bb) for bb in BrkSymbolDic]
    brokerDb.save()

#broker.objects

In [15]:
obj = BrokerDB.objects()[0]
print(BrokerDB.objects.count())

# test find max count of call symbols at sample time window
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$unwind": "$position" },
    {"$group": { "_id": "$position.symbol", "sum_puts": { "$sum": "$position.puts" }, "sum_calls": { "$sum": "$position.calls" } }  },
    {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol

10
{'_id': 6116, 'sum_puts': 2794, 'sum_calls': 2696}
{'_id': 2603, 'sum_puts': 696, 'sum_calls': 1008}
{'_id': 2610, 'sum_puts': 784, 'sum_calls': 662}


6116

In [16]:
# test find symbol's call event at sample time window
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$unwind": "$position" },
    {"$match": { "position.symbol": MaxCallSymbol } },
    {"$sort": { "date": 1} }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
# put to abu backtest algorithm to test broker win rate
for it in items:
    print (it['date'])

2021-02-16 21:41:46.464000
2021-02-17 21:41:46.444000


In [17]:
# test find max time of call(at least Call 5units threshold) symbols at sample time window
# stage1
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$match": { "position.calls": { "$gte": 5} } },
    {"$match": { "position.puts": { "$lte": 5} } },
    {"$unwind": "$position" },
    {"$group": { "_id": "$position.symbol", "times": { "$sum": 1 } } },
    {"$sort":  { "times": -1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol


{'_id': 2313, 'times': 2}
{'_id': 3526, 'times': 2}
{'_id': 6282, 'times': 2}


2313

In [18]:
# test find max time of Daily punch(隔(n)日沖) symbols at sample time window
# stage0
pipeline = [
    {"$match": { "date": { "$lte": datetime.now() - timedelta(days=1)} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2)} } },
    {"$match": { "position.calls": { "$gte": 5} } },
    {"$match": { "position.puts": { "$gte": 5} } },
    {"$unwind": "$position" },
    {"$group": { "_id": "$position.symbol", "times": { "$sum": 1 } } },
    {"$sort":  { "times": -1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol


# stage1
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=1) } } },
    {"$unwind": "$position" },
    {"$match": { "_id": "$position.symbol" } },
    {"$sort":  { "times": -1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list

{'_id': 2313, 'times': 1}
{'_id': 3526, 'times': 1}
{'_id': 6282, 'times': 1}


In [19]:
import asyncio
import motor.motor_asyncio
from datetime import date, datetime
import pprint
import sys

sys.path.insert(0, os.path.abspath('../'))
import abupy
from abupy.UtilBu.ABuDateUtil import str_to_datetime

NumExpr defaulting to 8 threads.


In [20]:

client = motor.motor_asyncio.AsyncIOMotorClient('mongodb://localhost:27017')

In [24]:
db = client['symbol-db']
collection = db['StockHisTrader']
loop = asyncio.get_event_loop()

In [25]:
# spider cmd
#%scrapy crawl StockHisTraderSpider --logfile test.log
yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
dd = str_to_datetime(cdate)
symbol = '3617'

async def do_find_one():
    document = await db['StockHisTrader'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    

loop.run_until_complete(do_find_one())

{'_id': ObjectId('602d135f5e500f73b3d33d6e'),
 'date': datetime.datetime(2021, 2, 17, 0, 0),
 'symbol': '3617',
 'traders': [{'call': '6', 'put': '0', 'tdname': '大昌-新店', 'weight': '8.11'},
             {'call': '0', 'put': '12', 'tdname': '元大-新竹', 'weight': '16.22'},
             {'call': '5', 'put': '0', 'tdname': '國泰-館前', 'weight': '6.76'},
             {'call': '0', 'put': '4', 'tdname': '日盛-中壢', 'weight': '5.41'},
             {'call': '5', 'put': '0', 'tdname': '富邦-仁愛', 'weight': '6.76'},
             {'call': '0', 'put': '3', 'tdname': '中國信託-忠孝', 'weight': '4.05'},
             {'call': '4', 'put': '0', 'tdname': '土銀-建國', 'weight': '5.41'},
             {'call': '0', 'put': '3', 'tdname': '玉山-桃園', 'weight': '4.05'},
             {'call': '4', 'put': '0', 'tdname': '瑞士信貸', 'weight': '5.41'},
             {'call': '0', 'put': '3', 'tdname': '元大-景美', 'weight': '4.05'},
             {'call': '4', 'put': '1', 'tdname': '美林', 'weight': '4.05'},
             {'call': '0', 'put': '2', 't

In [39]:

async def do_find_max_calls_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
        {"$match": { "date": { "$lte": datetime.now()} } },
        {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
        {"$unwind": "$traders" },
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" } }  },
        {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": 3 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    async for cursor in collect: # clean cursor ptr and store to list
        pprint.pprint(cursor)
        

loop.run_until_complete(do_find_max_calls_symbol())        

{'_id': {'symbol': '912000', 'tdname': '凱基-大直'},
 'sum_calls': 780000.0,
 'sum_puts': 0.0}
{'_id': {'symbol': '00663L', 'tdname': '元大證券'},
 'sum_calls': 591000.0,
 'sum_puts': 630000.0}
{'_id': {'symbol': '00733', 'tdname': '富邦證券'},
 'sum_calls': 257000.0,
 'sum_puts': 144000.0}
